In [1]:
import numpy as np
import quadrotor

model = quadrotor.derive_model()

t = np.arange(0, 10, 0.001)
T_trim = 0.582324
u_sim = np.vstack(
    [
        1.2*T_trim*np.ones(len(t)),
        1.2*T_trim*np.ones(len(t)),
        T_trim*np.ones(len(t)),
        T_trim*np.ones(len(t)),
    ]
)

res = quadrotor.sim(
    model=model,
    t=t,
    u=u_sim,
    x0={
        "normalized_motor_0": T_trim,
        "normalized_motor_1": T_trim,
        "normalized_motor_2": T_trim,
        "normalized_motor_3": T_trim,
    },
)

quadrotor.plotting(model, t, res, './fig')

In [2]:
%%timeit
res = quadrotor.sim(
    model,
    t,
    u_sim,
    {
        "normalized_motor_0": T_trim,
        "normalized_motor_0": T_trim,
        "normalized_motor_0": T_trim,
        "normalized_motor_0": T_trim,
    },
)

47.2 ms ± 332 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
